In [5]:
include("../read.jl")
include("../constraints.jl")
include("../partitions.jl")
include("../problem.jl")
include("../active.jl")

jobs = build_jobs("/Users/jpoullet/Documents/MIT/Courses/15.094 Robust optimization/RobustProject/data/example.csv", [0, 0])
# jobs = jobs[2:3, :]
# jobs[:ID] = 1:nrow(jobs)

# Box constraints on z.
base = ConstraintSet()
add_constraint!(base, z -> norm(z, Inf) - 100)
add_constraint!(base, z -> norm(z, 1) - 100)

MethodError: MethodError: no method matching add_constraint!(::ConstraintSet, ::getfield(Main, Symbol("##39#40")))
Closest candidates are:
  add_constraint!(::ConstraintSet, !Matched::C<:Constraint) where C<:Constraint at /Users/jpoullet/Documents/MIT/Courses/15.094 Robust optimization/RobustProject/src/constraints.jl:67

In [216]:
# partitions = [base]

num_workers = 2
cost_backup = 10

# Get index parameters.
num_intervals = maximum(jobs[:INTERVAL])
num_jobs = nrow(jobs)
num_partitions = length(partitions)

# Big M parameter.
M = 1e9

# Create model.
m = RobustModel(solver=GurobiSolver(OutputFlag=0))

### Static

# Epigraph variable.
@variable(m, r)

# Assignment variables for workers to jobs.
@variable(m, x[1:num_workers, 1:num_jobs], Bin)

# Backup agent variables.
@variable(m, z[1:num_jobs], Bin)

## Adjustable

# Workers to jobs.
@variable(m, x_u[1:num_partitions, 1:num_workers, 2:num_intervals, 1:num_jobs], Bin)

# Backup agents.
@variable(m, z_u[1:num_partitions, 2:num_intervals, 1:num_jobs], Bin)

# Variables to capture clashes.
@variable(m, y_u[1:num_partitions, 1:num_jobs, 1:num_jobs], Bin)

## Uncertainty

# One uncertain vector for each partition.
@uncertain(m, u[1:num_partitions, 1:num_jobs])
for p in 1:num_partitions
    apply_constraints(m, u[p, :], base)
    apply_constraints(m, u[p, :], partitions[p])
end

In [217]:
initial_jobs = jobs_not_in_interval(jobs, 1)
@constraint(m, z[initial_jobs] .== 0)

for i in 1:num_workers

    # Jobs in interval 1 are static.
    @constraint(m, x[i, initial_jobs] .== 0)    

    # Remaining jobs are adjustable.
    for t in 2:num_intervals, p in 1:num_partitions
        interval_jobs = jobs_not_in_interval(jobs, t)
        @constraint(m, x_u[p, i, t, :][interval_jobs] .== 0)
        @constraint(m, z_u[p, t, :][interval_jobs] .== 0)
    end
end

In [218]:
# Objective function.
@objective(m, Min, r + cost_backup * sum(z[jobs_in_interval(jobs, 1)]))
for p in 1:num_partitions
    @constraint(m,  r >= cost_backup * sum(sum(z_u[p, t, :][jobs_in_interval(jobs, t)] for t in 2:num_intervals)))
end

In [219]:
# Constraints covertness t = 1.
for j in jobs_in_interval(jobs, 1)
    @constraint(m, sum(x[i, j] for i in 1:num_workers) + z[j] == 1)
end

In [220]:
# Constraints covertness t > 1.
for t in 2:num_intervals, p in 1:num_partitions
    for j in jobs_in_interval(jobs, t)
        @constraint(m, sum(x_u[p, i, t, j] for i in 1:num_workers) + z_u[p, t, j] == 1)
    end
end

In [222]:
# Constraint chaining. Need to check splitting of uncertainty.
for j in 1:num_jobs, k in 1:num_jobs, p in 1:num_partitions
    if j != k
        j_end = jobs[j, :END]
        k_start = jobs[k, :START]
        @constraint(m, -M * y_u[p, j, k] <= k_start + u[p, k] - (j_end + u[p, j]))
        # @constraint(m, k_start + u[k] - (j_end + u[j]) <= M * (1 - y_u[j, k]))
    end
end

# start_1 = jobs[1, :START]
# start_2 = jobs[2, :START]
# end_1 = jobs[1, :END]
# end_2 = jobs[2, :END]
# @constraint(m, -M * y_u[1, 1, 2] <= start_2 + u[1, 2] - (end_1 + u[1, 1]))
# @constraint(m, -M * y_u[1, 2, 1] <= start_1 + u[1, 1] - (end_2 + u[1, 2]))

In [223]:
# No clash for initial jobs with any other jobs.
for i in 1:num_workers
    for j in 1:num_jobs, k in (j + 1):num_jobs, p in 1:num_partitions

        # Clashes of initial jobs with other initial jobs.
        @constraint(m, x[i, j] + x[i, k] <= 3 - (y_u[p, j, k] + y_u[p, k, j]))

        # Clashes of initial jobs with adjustable jobs.
        for t in 2:num_intervals
            @constraint(m, x[i, j] + x_u[p, i, t, k] <= 3 - (y_u[p, j, k] + y_u[p, k, j]))
        end
    end
end

In [224]:
# No clash for adjustable jobs.
for i in 1:num_workers
    for t in 2:num_intervals, t_bar in t:num_intervals, p in 1:num_partitions
        for j in 1:num_jobs, k in (j + 1):num_jobs
            @constraint(m, x_u[p, i, t, j] + x_u[p, i, t_bar, k] <= 3 - (y_u[p, j, k] + y_u[p, k, j]))
        end
    end
end

In [225]:
solve(m)

Academic license - for non-commercial use only


:Optimal

In [226]:
getobjectivevalue(m)

0.0

In [179]:
getvalue(y_u)

1×3×3 Array{Float64,3}:
[:, :, 1] =
 -0.0  1.0  1.0

[:, :, 2] =
 1.0  -0.0  1.0

[:, :, 3] =
 1.0  1.0  -0.0

In [180]:
getvalue(x_u)

x_u: 4 dimensions:
[1,:,:,:]
  [1,1,:,:]
    [1,1,2,:]
      [1,1,2,1] = 0.0
      [1,1,2,2] = 1.0
      [1,1,2,3] = -0.0
  [1,2,:,:]
    [1,2,2,:]
      [1,2,2,1] = 0.0
      [1,2,2,2] = -0.0
      [1,2,2,3] = 1.0

In [181]:
getvalue(z_u)

z_u: 3 dimensions:
[1,:,:]
  [1,2,:]
    [1,2,1] = 1.0
    [1,2,2] = -0.0
    [1,2,3] = -0.0

In [182]:
z, z_u, x, x_u, y_u, r = getvalue(z), getvalue(z_u), getvalue(x), getvalue(x_u), getvalue(y_u), getvalue(r)

([-0.0, -0.0, -0.0], z_u: 3 dimensions:
[1,:,:]
  [1,2,:]
    [1,2,1] = 1.0
    [1,2,2] = -0.0
    [1,2,3] = -0.0, [-0.0 -0.0 -0.0; -0.0 -0.0 -0.0], x_u: 4 dimensions:
[1,:,:,:]
  [1,1,:,:]
    [1,1,2,:]
      [1,1,2,1] = 0.0
      [1,1,2,2] = 1.0
      [1,1,2,3] = -0.0
  [1,2,:,:]
    [1,2,2,:]
      [1,2,2,1] = 0.0
      [1,2,2,2] = -0.0
      [1,2,2,3] = 1.0, [-0.0 1.0 1.0]

[1.0 -0.0 1.0]

[1.0 1.0 -0.0], 10.0)

In [199]:
points = []

for (p, partition) in enumerate(partitions)

    partition_points = []

    for j in 1:num_jobs, k in 1:num_jobs
        
        if j != k
            m = Model(solver=GurobiSolver(OutputFlag=0))

            @variable(m, -100 <= u[1:num_jobs] <= 100)
            @variable(m, v[1:num_jobs])
            @constraint(m, v .>= u)
            @constraint(m, v .>= -u)
            @constraint(m, sum(v) <= 100)

            j_end = jobs[j, :END]
            k_start = jobs[k, :START]

            @objective(m, Min, k_start + u[k] - (j_end + u[j]))

            solve(m)
            if getobjectivevalue(m) < 0
                push!(partition_points, getvalue(u))
            end
        end
    end

    push!(points, transpose(hcat(partition_points...)))

end

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only


In [210]:
p = points


1-element Array{Any,1}:
 [0.0 -100.0 0.0; 0.0 0.0 -100.0; … ; 0.0 0.0 100.0; 0.0 0.0 100.0]

In [211]:
points = points[1]

6×3 Transpose{Float64,Array{Float64,2}}:
 0.0  -100.0     0.0
 0.0     0.0  -100.0
 0.0   100.0     0.0
 0.0     0.0  -100.0
 0.0     0.0   100.0
 0.0     0.0   100.0

In [214]:
function max_margin(x, y)

    # Compute value of affine function at midpoint between x and y.
    alpha = dot(y - x, 0.5 * (x + y))

    # Return function in the required form.
    return z -> dot(y - x, z) - alpha
end

# Get number of points and dimension of space.
m, n = size(points)

partitions = [ConstraintSet() for i in 1:m]

for i in 1:(m - 1), j in (i + 1):m
        
    x = points[i, :]
    y = points[j, :]

    add_constraint!(partitions[i], max_margin(x, y))
    add_constraint!(partitions[j], max_margin(y, x))

end

In [215]:
partitions

6-element Array{ConstraintSet,1}:
 ConstraintSet(Function[##205#206{Array{Float64,1},Array{Float64,1},Float64}([0.0, -100.0, 0.0], [0.0, 0.0, -100.0], 0.0), ##205#206{Array{Float64,1},Array{Float64,1},Float64}([0.0, -100.0, 0.0], [0.0, 100.0, 0.0], 0.0), ##205#206{Array{Float64,1},Array{Float64,1},Float64}([0.0, -100.0, 0.0], [0.0, 0.0, -100.0], 0.0), ##205#206{Array{Float64,1},Array{Float64,1},Float64}([0.0, -100.0, 0.0], [0.0, 0.0, 100.0], 0.0), ##205#206{Array{Float64,1},Array{Float64,1},Float64}([0.0, -100.0, 0.0], [0.0, 0.0, 100.0], 0.0)])
 ConstraintSet(Function[##205#206{Array{Float64,1},Array{Float64,1},Float64}([0.0, 0.0, -100.0], [0.0, -100.0, 0.0], 0.0), ##205#206{Array{Float64,1},Array{Float64,1},Float64}([0.0, 0.0, -100.0], [0.0, 100.0, 0.0], 0.0), ##205#206{Array{Float64,1},Array{Float64,1},Float64}([0.0, 0.0, -100.0], [0.0, 0.0, -100.0], 0.0), ##205#206{Array{Float64,1},Array{Float64,1},Float64}([0.0, 0.0, -100.0], [0.0, 0.0, 100.0], 0.0), ##205#206{Array{Float64,1},Arra

In [221]:
num_partitions

6

In [37]:
include("../read.jl")
include("../constraints.jl")
include("../partitions.jl")
include("../problem.jl")
include("../active.jl")

jobs = build_jobs("/Users/jpoullet/Documents/MIT/Courses/15.094 Robust optimization/RobustProject/data/example.csv", [0, 0])
problem = Problem(jobs, 2, 10, 1e9)



Problem(3×9 DataFrame. Omitted printing of 2 columns
│ Row │ DATE     │ FLIGHT_NUM │ WAY     │ TEAM    │ NUMBER │ START   │ END     │
│     │ Int64⍰   │ Int64⍰     │ String⍰ │ String⍰ │ Int64⍰ │ Float64 │ Float64 │
├─────┼──────────┼────────────┼─────────┼─────────┼────────┼─────────┼─────────┤
│ 1   │ 20170620 │ 1          │ Arrival │ ASA     │ 3      │ 50.0    │ 150.0   │
│ 2   │ 20170620 │ 2          │ Arrival │ POLY    │ 2      │ 233.333 │ 350.0   │
│ 3   │ 20170620 │ 3          │ Arrival │ POLY    │ 1      │ 233.333 │ 350.0   │, 2, 10.0, 1.0e9, 3, 2)

In [7]:
using JuMP, JuMPeR, Gurobi, LinearAlgebra

In [33]:
function solve_partition_modelC(problem::Problem, base::ConstraintSet, partitions::Array{ConstraintSet})

    # Get problem data.
    num_partitions = length(partitions)
    jobs = problem.jobs
    num_jobs = problem.num_jobs
    num_workers = problem.num_workers
    num_intervals = problem.num_intervals
    cost_backup = problem.cost_backup
    M = problem.M

    # Create model.
    m = RobustModel(solver=GurobiSolver(OutputFlag=0))

    ### Static
    
    ## Uncertainty

    # One uncertain vector for each partition.
    @variable(m, u[1:num_partitions, 1:num_jobs])
    for p in 1:num_partitions
        apply_constraints!(m, u[p, :], base)
        apply_constraints!(m, u[p, :], partitions[p])
    end
    
    for con in m.linconstr
        println(con)
    end
    # Epigraph variable.
    @variable(m, r)

    # Assignment variables for workers to jobs.
    @variable(m, x[1:num_workers, 1:num_jobs], Bin)

    # Backup agent variables.
    @variable(m, z[1:num_jobs], Bin)

    ## Adjustable

    # Workers to jobs.
    @variable(m, x_u[1:num_partitions, 1:num_workers, 2:num_intervals, 1:num_jobs], Bin)

    # Backup agents.
    @variable(m, z_u[1:num_partitions, 2:num_intervals, 1:num_jobs], Bin)

    # Variables to capture clashes.
    @variable(m, y_u[1:num_partitions, 1:num_jobs, 1:num_jobs], Bin)
 

    # Constraint on the tasks not assigned during interval t.
    initial_jobs = jobs_not_in_interval(jobs, 1)
    @constraint(m, z[initial_jobs] .== 0)
    for i in 1:num_workers

        # Jobs in interval 1 are static.
        @constraint(m, x[i, initial_jobs] .== 0)

        # Remaining jobs are adjustable.
        for t in 2:num_intervals, p in 1:num_partitions
            interval_jobs = jobs_not_in_interval(jobs, t)
            @constraint(m, x_u[p, i, t, :][interval_jobs] .== 0)
            @constraint(m, z_u[p, t, :][interval_jobs] .== 0)
        end
    end

    # Objective function.
    @objective(m, Min, r + cost_backup * sum(z[jobs_in_interval(jobs, 1)]))
    for p in 1:num_partitions
        @constraint(m,  r >= cost_backup * sum(sum(z_u[p, t, :][jobs_in_interval(jobs, t)] for t in 2:num_intervals)))
    end

    # Convering constraints for first stage.
    for j in jobs_in_interval(jobs, 1)
        @constraint(m, sum(x[i, j] for i in 1:num_workers) + z[j] == 1)
    end

    # Covering constraints for adjustable stages.
    for t in 2:num_intervals, p in 1:num_partitions
        for j in jobs_in_interval(jobs, t)
            @constraint(m, sum(x_u[p, i, t, j] for i in 1:num_workers) + z_u[p, t, j] == 1)
        end
    end

    # Constraint chaining.
    for j in 1:num_jobs, k in 1:num_jobs, p in 1:num_partitions
        if j != k
            j_end = jobs[j, :END]
            k_start = jobs[k, :START]
            @constraint(m, - M * y_u[p, j, k] <= k_start + u[p, k] - (j_end + u[p, j]))
            # @constraint(m, k_start + u[k] - (j_end + u[j]) <= M * (1 - y_u[j, k]))
        end
    end

    # No clash for initial jobs with any other jobs.
    for i in 1:num_workers
        for j in 1:num_jobs, k in (j + 1):num_jobs, p in 1:num_partitions

            # Clashes of initial jobs with other initial jobs.
            @constraint(m, x[i, j] + x[i, k] <= 3 - (y_u[p, j, k] + y_u[p, k, j]))

            # Clashes of initial jobs with adjustable jobs.
            for t in 2:num_intervals
                @constraint(m, x[i, j] + x_u[p, i, t, k] <= 3 - (y_u[p, j, k] + y_u[p, k, j]))
            end
        end
    end

    # No clash for adjustable jobs.
    for i in 1:num_workers
        for t in 2:num_intervals, t_bar in t:num_intervals, p in 1:num_partitions
            for j in 1:num_jobs, k in (j + 1):num_jobs
                @constraint(m, x_u[p, i, t, j] + x_u[p, i, t_bar, k] <= 3 - (y_u[p, j, k] + y_u[p, k, j]))
            end
        end
    end
   
    solve(m)
    
    return m
end

solve_partition_modelC (generic function with 1 method)

In [88]:
function solve_partition_model(problem::Problem, base::ConstraintSet, partitions::Array{ConstraintSet})

    # Get problem data.
    num_partitions = length(partitions)
    jobs = problem.jobs
    num_jobs = problem.num_jobs
    num_workers = problem.num_workers
    num_intervals = problem.num_intervals
    cost_backup = problem.cost_backup
    M = 10000

    # Create model.
    m = RobustModel(solver=GurobiSolver(OutputFlag=0))

    ### Static

    # Epigraph variable.
    @variable(m, r)

    # Assignment variables for workers to jobs.
    @variable(m, x[1:num_workers, 1:num_jobs], Bin)

    # Backup agent variables.
    @variable(m, z[1:num_jobs], Bin)

    ## Adjustable

    # Workers to jobs.
    @variable(m, x_u[1:num_partitions, 1:num_workers, 2:num_intervals, 1:num_jobs], Bin)

    # Backup agents.
    @variable(m, z_u[1:num_partitions, 2:num_intervals, 1:num_jobs], Bin)

    # Variables to capture clashes.
    @variable(m, y_u[1:num_partitions, 1:num_jobs, 1:num_jobs], Bin)
 
    ## Uncertainty

    # One uncertain vector for each partition.
    @uncertain(m, u[1:num_partitions, 1:num_jobs])
    for p in 1:num_partitions
        apply_constraints!(m, u[p, :], base)
        apply_constraints!(m, u[p, :], partitions[p])
    end

    # Constraint on the tasks not assigned during interval t.
    initial_jobs = jobs_not_in_interval(jobs, 1)
    @constraint(m, z[initial_jobs] .== 0)
    for i in 1:num_workers

        # Jobs in interval 1 are static.
        @constraint(m, x[i, initial_jobs] .== 0)

        # Remaining jobs are adjustable.
        for t in 2:num_intervals, p in 1:num_partitions
            interval_jobs = jobs_not_in_interval(jobs, t)
            @constraint(m, x_u[p, i, t, :][interval_jobs] .== 0)
            @constraint(m, z_u[p, t, :][interval_jobs] .== 0)
        end
    end

    # Objective function.
    @objective(m, Min, r + cost_backup * sum(z[jobs_in_interval(jobs, 1)]))
    for p in 1:num_partitions
        @constraint(m,  r >= cost_backup * sum(sum(z_u[p, t, :][jobs_in_interval(jobs, t)] for t in 2:num_intervals)))
    end

    # Convering constraints for first stage.
    for j in jobs_in_interval(jobs, 1)
        @constraint(m, sum(x[i, j] for i in 1:num_workers) + z[j] == 1)
    end

    # Covering constraints for adjustable stages.
    for t in 2:num_intervals, p in 1:num_partitions
        for j in jobs_in_interval(jobs, t)
            @constraint(m, sum(x_u[p, i, t, j] for i in 1:num_workers) + z_u[p, t, j] == 1)
        end
    end

    # Constraint chaining.
    for j in 1:num_jobs, k in 1:num_jobs, p in 1:num_partitions
        if j != k
            j_end = jobs[j, :END]
            k_start = jobs[k, :START]
            @constraint(m, - M * y_u[p, j, k] <= k_start + u[p, k] - (j_end + u[p, j]))
            # @constraint(m, k_start + u[k] - (j_end + u[j]) <= M * (1 - y_u[j, k]))
        end
    end

    # No clash for initial jobs with any other jobs.
    for i in 1:num_workers
        for j in 1:num_jobs, k in (j + 1):num_jobs, p in 1:num_partitions

            # Clashes of initial jobs with other initial jobs.
            @constraint(m, x[i, j] + x[i, k] <= 3 - (y_u[p, j, k] + y_u[p, k, j]))

            # Clashes of initial jobs with adjustable jobs.
            for t in 2:num_intervals
                @constraint(m, x[i, j] + x_u[p, i, t, k] <= 3 - (y_u[p, j, k] + y_u[p, k, j]))
            end
        end
    end

    # No clash for adjustable jobs.
    for i in 1:num_workers
        for t in 2:num_intervals, t_bar in t:num_intervals, p in 1:num_partitions
            for j in 1:num_jobs, k in (j + 1):num_jobs
                @constraint(m, x_u[p, i, t, j] + x_u[p, i, t_bar, k] <= 3 - (y_u[p, j, k] + y_u[p, k, j]))
            end
        end
    end

    solve(m)
    
    return m
end

solve_partition_model (generic function with 1 method)

In [89]:
function solve_lower_bound_model(problem::Problem, points::Array{<:Number})::Model

    # Get problem data.
    num_points = length(points[:,1])
    jobs = problem.jobs
    num_jobs = problem.num_jobs
    num_workers = problem.num_workers
    num_intervals = problem.num_intervals
    cost_backup = problem.cost_backup
    M = problem.M

    # Create model.
    m = Model(solver=GurobiSolver(OutputFlag=0))

    ### Static

    # Epigraph variable.
    @variable(m, r)

    # Assignment variables for workers to jobs.
    @variable(m, x[1:num_workers, 1:num_jobs], Bin)

    # Backup agent variables.
    @variable(m, z[1:num_jobs], Bin)

    ## Adjustable

    # Workers to jobs.
    @variable(m, x_u[1:num_points,1:num_workers, 2:num_intervals, 1:num_jobs], Bin)

    # Backup agents.
    @variable(m, z_u[1:num_points, 2:num_intervals, 1:num_jobs], Bin)

    # Variables to capture clashes.
    @variable(m, y_u[1:num_points, 1:num_jobs, 1:num_jobs], Bin)

    # Constraint on the tasks not assigned during interval t.
    initial_jobs = jobs_not_in_interval(jobs, 1)
    @constraint(m, z[initial_jobs] .== 0)
    for i in 1:num_workers

        # Jobs in interval 1 are static.
        @constraint(m, x[i, initial_jobs] .== 0)

        # Remaining jobs are adjustable.
        for t in 2:num_intervals, p in 1:num_points
            interval_jobs = jobs_not_in_interval(jobs, t)
            @constraint(m, x_u[p, i, t, :][interval_jobs] .== 0)
            @constraint(m, z_u[p, t, :][interval_jobs] .== 0)
        end
    end

    # Objective function.
    @objective(m, Min, r + cost_backup * sum(z[jobs_in_interval(jobs, 1)]))
    for p in 1:num_points
        @constraint(m,  r >= cost_backup * sum(sum(z_u[p, t, :][jobs_in_interval(jobs, t)] for t in 2:num_intervals)))
    end

    # Convering constraints for first stage.
    for j in jobs_in_interval(jobs, 1)
        @constraint(m, sum(x[i, j] for i in 1:num_workers) + z[j] == 1)
    end

    # Covering constraints for adjustable stages.
    for t in 2:num_intervals, p in 1:num_points
        for j in jobs_in_interval(jobs, t)
            @constraint(m, sum(x_u[p, i, t, j] for i in 1:num_workers) + z_u[p, t, j] == 1)
        end
    end

    # Constraint chaining.
    for j in 1:num_jobs, k in 1:num_jobs, p in 1:num_points
        if j != k
            j_end = jobs[j, :END]
            k_start = jobs[k, :START]
            @constraint(m, - M * y_u[p, j, k] <= k_start + points[p, k] - (j_end + points[p, j]))
            # @constraint(m, k_start + u[k] - (j_end + u[j]) <= M * (1 - y_u[j, k]))
        end
    end

    # No clash for initial jobs with any other jobs.
    for i in 1:num_workers
        for j in 1:num_jobs, k in (j + 1):num_jobs, p in 1:num_points

            # Clashes of initial jobs with other initial jobs.
            @constraint(m, x[i, j] + x[i, k] <= 3 - (y_u[p, j, k] + y_u[p, k, j]))

            # Clashes of initial jobs with adjustable jobs.
            for t in 2:num_intervals
                @constraint(m, x[i, j] + x_u[p, i, t, k] <= 3 - (y_u[p, j, k] + y_u[p, k, j]))
            end
        end
    end

    # No clash for adjustable jobs.
    for i in 1:num_workers
        for t in 2:num_intervals, t_bar in t:num_intervals, p in 1:num_points
            for j in 1:num_jobs, k in (j + 1):num_jobs
                @constraint(m, x_u[p, i, t, j] + x_u[p, i, t_bar, k] <= 3 - (y_u[p, j, k] + y_u[p, k, j]))
            end
        end
    end

    solve(m)
    return m
end

solve_lower_bound_model (generic function with 1 method)

In [90]:
base = ConstraintSet()
add_constraint!(base, GeneralConstraint(u -> u - 100))
add_constraint!(base, GeneralConstraint(u -> -u - 100))
add_constraint!(base, NormConstraint(1, 200))

# No other constraints in the single partition to start with.
partitions = [ConstraintSet()]
solved = solve_partition_model(problem, base, partitions)
println(getobjectivevalue(solved))

solved = solve_partition_model(problem, base, partitions)
println(getobjectivevalue(solved))

points = basic(solved, problem, base, partitions)

partitions = voronoi(points[1])

stupidSolved = solve_stupid_model(problem, base, partitions)
println(getobjectivevalue(stupidSolved))

solved = solve_partition_model(problem, base, partitions)
println(getobjectivevalue(solved))
@show(getvalue(getindex(solved,:x_u)))
@show(getvalue(getindex(solved,:y_u)))

solved = solve_partition_modelC(problem, base, partitions)
println(getobjectivevalue(solved))

lwsolved= solve_lower_bound_model(problem,points[1])
println(getobjectivevalue(lwsolved))

partitions = [ConstraintSet()]
solved = solve_partition_model(problem, base, partitions)
println(getobjectivevalue(solved))

Academic license - for non-commercial use only
10.0
Academic license - for non-commercial use only
10.0
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
10.0
Academic license - for non-commercial use only
10.0
getvalue(getindex(solved, :x_u)) = x_u: 4 dimensions:
[1,:,:,:]
  [1,1,:,:]
    [1,1,2,:]
      [1,1,2,1] = 1.0
      [1,1,2,2] = 0.0
      [1,1,2,3] = 0.0
  [1,2,:,:]
    [1,2,2,:]
      [1,2,2,1] = 0.0
      [1,2,2,2] = 0.0
      [1,2,2,3] = 1.0
[2,:,:,:]
  [2,1,:,:]
    [2,1,2,:]
      [2,1,2,1] = 0.0
      [2,1,2,2] = 0.0
      [2,1,2,3] = 1.0
  [2,2,:,:]
    [2,

In [86]:
function solve_stupid_model(problem::Problem, base::ConstraintSet, partitions::Array{ConstraintSet})

    # Get problem data.
    num_partitions = length(partitions)
    jobs = problem.jobs
    num_jobs = problem.num_jobs
    num_workers = problem.num_workers
    num_intervals = problem.num_intervals
    cost_backup = problem.cost_backup
    M = 1000
    jobsStart = [50,233,233]
    jobsEnd = [150,350,350]

    # Create model.
    m = RobustModel(solver=GurobiSolver(OutputFlag=0))

    ### Static

    # Epigraph variable.
    @variable(m, r)

    # Assignment variables for workers to jobs.
    @variable(m, x[1:num_workers, 1:num_jobs], Bin)

    # Backup agent variables.
    @variable(m, z[1:num_jobs], Bin)

    ## Adjustable

    # Workers to jobs.
    @variable(m, x_u[1:num_partitions, 1:num_workers, 2:num_intervals, 1:num_jobs], Bin)

    # Backup agents.
    @variable(m, z_u[1:num_partitions, 2:num_intervals, 1:num_jobs], Bin)

    # Variables to capture clashes.
    @variable(m, y_u[1:num_partitions, 1:num_jobs, 1:num_jobs], Bin)
 
    ## Uncertainty

    # One uncertain vector for each partition.
    @uncertain(m, u1[1:num_jobs])
    @constraint(m, norm(u1,1) <= 200)
    @constraint(m, norm(u1,Inf) <= 100)
    #apply_constraints!(m, u1, base)
    #apply_constraints!(m, u1, partitions[1])
    @constraint(m, 2*u1[1] - u1[2] <=50)
    
    @uncertain(m, u2[1:num_jobs])
    @constraint(m, norm(u2,1) <= 200)
    @constraint(m, norm(u2,Inf) <= 100)
    #apply_constraints!(m, u2, base)
    #apply_constraints!(m, u2, partitions[2])
    @constraint(m, -2*u1[1] + u1[2] <=-50)


    # Constraint on the tasks not assigned during interval t.
    initial_jobs = jobs_not_in_interval(jobs, 1)
    @constraint(m, z[initial_jobs] .== 0)
    for i in 1:num_workers

        # Jobs in interval 1 are static.
        @constraint(m, x[i, initial_jobs] .== 0)

        # Remaining jobs are adjustable.
        for t in 2:num_intervals, p in 1:num_partitions
            interval_jobs = jobs_not_in_interval(jobs, t)
            @constraint(m, x_u[p, i, t, :][interval_jobs] .== 0)
            @constraint(m, z_u[p, t, :][interval_jobs] .== 0)
        end
    end

    # Objective function.
    @objective(m, Min, r + cost_backup * sum(z[j] for j in jobs_in_interval(jobs, 1)))
    for p in 1:num_partitions
        @constraint(m,  r >= cost_backup * sum(sum(z_u[p, t, :][jobs_in_interval(jobs, t)] for t in 2:num_intervals)))
    end

    # Convering constraints for first stage.
    for j in jobs_in_interval(jobs, 1)
        @constraint(m, sum(x[i, j] for i in 1:num_workers) + z[j] == 1)
    end

    # Covering constraints for adjustable stages.
    for t in 2:num_intervals, p in 1:num_partitions
        for j in jobs_in_interval(jobs, t)
            @constraint(m, sum(x_u[p, i, t, j] for i in 1:num_workers) + z_u[p, t, j] == 1)
        end
    end

    # Constraint chaining.
    for j in 1:num_jobs, k in 1:num_jobs
        if j != k
            j_end = jobsEnd[j]
            k_start = jobsStart[k]
            @constraint(m, - M * y_u[1, j, k] <= k_start + u1[k] - (j_end + u1[j]))
            # @constraint(m, k_start + u[k] - (j_end + u[j]) <= M * (1 - y_u[j, k]))
        end
    end
    
    # Constraint chaining.
    for j in 1:num_jobs, k in 1:num_jobs
        if j != k
            j_end = jobsEnd[j]
            k_start = jobsStart[k]
            @constraint(m, - M * y_u[2, j, k] <= k_start + u2[k] - (j_end + u2[j]))
            # @constraint(m, k_start + u[k] - (j_end + u[j]) <= M * (1 - y_u[j, k]))
        end
    end

    # No clash for initial jobs with any other jobs.
    for i in 1:num_workers
        for j in 1:num_jobs, k in (j + 1):num_jobs, p in 1:num_partitions

            # Clashes of initial jobs with other initial jobs.
            @constraint(m, x[i, j] + x[i, k] <= 3 - (y_u[p, j, k] + y_u[p, k, j]))

            # Clashes of initial jobs with adjustable jobs.
            for t in 2:num_intervals
                @constraint(m, x[i, j] + x_u[p, i, t, k] <= 3 - (y_u[p, j, k] + y_u[p, k, j]))
            end
        end
    end

    # No clash for adjustable jobs.
    for i in 1:num_workers
        for t in 2:num_intervals, t_bar in t:num_intervals, p in 1:num_partitions
            for j in 1:num_jobs, k in (j + 1):num_jobs
                @constraint(m, x_u[p, i, t, j] + x_u[p, i, t_bar, k] <= 3 - (y_u[p, j, k] + y_u[p, k, j]))
            end
        end
    end

    solve(m)
    
    return m
end

solve_stupid_model (generic function with 1 method)